# Optimize The Model

Using your knowledge of TensorFlow, optimize your model to achieve a target predictive accuracy higher than 75%.

Use any or all of the following methods to optimize your model:

Adjust the input data to ensure that no variables or outliers are causing confusion in the model, such as:
Dropping more or fewer columns.
Creating more bins for rare occurrences in columns.
Increasing or decreasing the number of values for each bin.
Add more neurons to a hidden layer.
Add more hidden layers.
Use different activation functions for the hidden layers.
Add or reduce the number of epochs to the training regimen.

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME"],axis=1)

application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_type_counts = application_df['APPLICATION_TYPE'].value_counts()

application_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_type_counts[application_type_counts < 1000].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df['CLASSIFICATION'].value_counts()

classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts>1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 500].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [9]:
#check DataFrame to make sure EIN and NAME are not there anymore, 
#plus the APPLICATION_TYPE AND CLASSIFICATION columns have cutoff values
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,Other,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies_df = pd.get_dummies(application_df).astype(int)
application_dummies_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [11]:
# Split our preprocessed data into our features and target arrays
X = application_dummies_df.drop(["IS_SUCCESSFUL"], axis='columns').values
y = application_dummies_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

***
# 1st Try Model

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# 1st model: layer1 -> 10, layer2 -> 20
input_features = len(X_train_scaled[0])
hidden_nodes_1st_try_layer1 =  10
hidden_nodes_1st_try_layer2 = 20
#output_nodes_layer = 1

nn_model_1st_try = tf.keras.models.Sequential()

# First hidden layer
nn_model_1st_try.add(tf.keras.layers.Dense(input_dim=input_features, units=hidden_nodes_1st_try_layer1 , activation="tanh"))

# Second hidden layer
nn_model_1st_try.add(tf.keras.layers.Dense(units=hidden_nodes_1st_try_layer2, activation="sigmoid"))

# Output layer
nn_model_1st_try.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model_1st_try.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                420       
                                                                 
 dense_1 (Dense)             (None, 20)                220       
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 661 (2.58 KB)
Trainable params: 661 (2.58 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model 1st try!
nn_model_1st_try.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model 1st try!
fit_model = nn_model_1st_try.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 0s 374us/step - loss: 0.5989 - accuracy: 0.7029
Epoch 2/100
804/804 [==============================] - 0s 404us/step - loss: 0.5673 - accuracy: 0.7269
Epoch 3/100
804/804 [==============================] - 0s 367us/step - loss: 0.5616 - accuracy: 0.7292
Epoch 4/100
804/804 [==============================] - 0s 365us/step - loss: 0.5584 - accuracy: 0.7304
Epoch 5/100
804/804 [==============================] - 0s 351us/step - loss: 0.5565 - accuracy: 0.7302
Epoch 6/100
804/804 [==============================] - 0s 368us/step - loss: 0.5549 - accuracy: 0.7318
Epoch 7/100
804/804 [==============================] - 0s 356us/step - loss: 0.5540 - accuracy: 0.7318
Epoch 8/100
804/804 [==============================] - 0s 355us/step - loss: 0.5529 - accuracy: 0.7321
Epoch 9/100
804/804 [==============================] - 0s 358us/step - loss: 0.5522 - accuracy: 0.7320
Epoch 10/100
804/804 [==============================] - 0s 360us/step - l

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_1st_try.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5564 - accuracy: 0.7236 - 128ms/epoch - 478us/step
Loss: 0.5564470291137695, Accuracy: 0.7236151695251465


In [17]:
# Export our model to HDF5 file
nn_model_1st_try.save("Resources/AlphabetSoupCharity_Optimization_1st_try.h5")

/Users/rolandabellano/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# 1st Try Model
The optimization of the model on the 1st try did not yeild any significant difference. The accuracy is still hovering at 73% 

1st Try elements:

1. Nothing changed in the features and target
2. hidden_nodes_1st_try_layer1 =  10
3. hidden_nodes_1st_try_layer2 = 20

***

# 2nd Try

In [18]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
# 1st model: layer1 -> 10, layer2 -> 20
input_features = len(X_train_scaled[0])
hidden_nodes_2nd_try_layer1 =  50
hidden_nodes_2nd_try_layer2 = 100
output_nodes_2nd_try_layer = 1

nn_model_2nd_try = tf.keras.models.Sequential()

# First hidden layer
nn_model_2nd_try.add(tf.keras.layers.Dense(input_dim=input_features, units=hidden_nodes_2nd_try_layer1 , activation="relu"))

# Second hidden layer
nn_model_2nd_try.add(tf.keras.layers.Dense(units=hidden_nodes_2nd_try_layer2, activation="sigmoid"))

# Output layer
nn_model_2nd_try.add(tf.keras.layers.Dense(units=output_nodes_2nd_try_layer, activation="sigmoid"))

# Check the structure of the model
nn_model_2nd_try.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 50)                2100      
                                                                 
 dense_4 (Dense)             (None, 100)               5100      
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 7301 (28.52 KB)
Trainable params: 7301 (28.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# Compile the model 2nd try!
nn_model_2nd_try.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [20]:
# Train the model 2nd try!
fit_model = nn_model_2nd_try.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 400us/step - loss: 0.5800 - accuracy: 0.7159
Epoch 2/100
804/804 [==============================] - 0s 391us/step - loss: 0.5608 - accuracy: 0.7289
Epoch 3/100
804/804 [==============================] - 0s 414us/step - loss: 0.5575 - accuracy: 0.7289
Epoch 4/100
804/804 [==============================] - 0s 389us/step - loss: 0.5564 - accuracy: 0.7293
Epoch 5/100
804/804 [==============================] - 0s 389us/step - loss: 0.5552 - accuracy: 0.7314
Epoch 6/100
804/804 [==============================] - 0s 391us/step - loss: 0.5540 - accuracy: 0.7308
Epoch 7/100
804/804 [==============================] - 0s 391us/step - loss: 0.5540 - accuracy: 0.7313
Epoch 8/100
804/804 [==============================] - 0s 390us/step - loss: 0.5527 - accuracy: 0.7309
Epoch 9/100
804/804 [==============================] - 0s 391us/step - loss: 0.5519 - accuracy: 0.7313
Epoch 10/100
804/804 [==============================] - 0s 390us/step - l

In [21]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model_2nd_try.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5588 - accuracy: 0.7254 - 116ms/epoch - 431us/step
Loss: 0.5588281154632568, Accuracy: 0.7253644466400146


In [22]:
# Export our model to HDF5 file
nn_model_2nd_try.save("Resources/AlphabetSoupCharity_Optimization_2nd_try.h5")

***
# 2nd Try Model

The optimization of the model on the 2nd try actually made the accuracy <b style="color:red">worse</b>. The accuracy dipped at 72%.

2nd Try elements:

1. Nothing changed in the features and target
2. hidden_nodes_2nd_try_layer1 =  50 with the activation=relu
3. hidden_nodes_2nd_try_layer2 = 100
4. output_nodes_2nd_try_layer = 1

###



***
# 3rd Model

## Let's take one column out instead of two. Let's take EIN and leave NAME alone to see if it makes a difference

In [23]:
#  Import and read the charity_data.csv
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [24]:
# Drop the non-beneficial ID columns, 'EIN'.
application_sans_EIN_df = application_df.drop(["EIN"],axis=1)

application_sans_EIN_df

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...
34294,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [25]:
# Determine the number of unique values in each column.
application_sans_EIN_df.nunique()

NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [26]:
# Count NAME instances in the column to see if there's a pattern
name_counts = application_df['NAME'].value_counts()
#  How many name counts are greater than 5?
name_counts[name_counts>5]

NAME
PARENT BOOSTER USA INC                             1260
TOPS CLUB INC                                       765
UNITED STATES BOWLING CONGRESS INC                  700
WASHINGTON STATE UNIVERSITY                         492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC     408
                                                   ... 
OLD OAK CLIFF CONSERVATION LEAGUE INC                 6
AMERICAN NEPHROLOGY NURSES ASSOCIATION                6
HUMBLE ISD EDUCATIONAL SUPPORT GROUPS INC             6
PROFESSIONAL LOADMASTER ASSOCIATION                   6
CBMC INC                                              6
Name: count, Length: 354, dtype: int64

### Look at the charity NAME(s). They appear more than once!

In [27]:
# Determine which values to replace if counts are less than or equal to 5.
names_to_replace = list(name_counts[name_counts <= 20].index)

# Replace in dataframe
for name_ in names_to_replace:
    application_sans_EIN_df['NAME'] = application_sans_EIN_df['NAME'].replace(name_,"Other")
    
# Check to make sure binning was successful
application_sans_EIN_df['NAME'].value_counts()

NAME
Other                                              22469
PARENT BOOSTER USA INC                              1260
TOPS CLUB INC                                        765
UNITED STATES BOWLING CONGRESS INC                   700
WASHINGTON STATE UNIVERSITY                          492
                                                   ...  
FULL GOSPEL BUSINESS MENS FELLOWSHIP IN AMERICA       21
ARIZONA FEDERATION OF GARDEN CLUBS                    21
ALPHA PHI SIGMA INC                                   21
AMERICAN IRIS SOCIETY                                 21
FRATERNAL ORDER OF POLICE                             21
Name: count, Length: 119, dtype: int64

In [28]:
# Let's big APPLICATION_TYPE
application_type_counts = application_sans_EIN_df['APPLICATION_TYPE'].value_counts()

application_type_counts

APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [29]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_type_counts[application_type_counts < 1000].index)

# Replace in dataframe
for app in application_types_to_replace:
    application_sans_EIN_df['APPLICATION_TYPE'] = application_sans_EIN_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_sans_EIN_df['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: count, dtype: int64

In [30]:
application_sans_EIN_df

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,Other,Other,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,Other,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,Other,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,Other,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,Other,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...,...
34294,Other,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,Other,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,Other,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [31]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_sans_EIN_df['CLASSIFICATION'].value_counts()

classification_counts

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: count, Length: 71, dtype: int64

In [32]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_counts[classification_counts>1]

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: count, dtype: int64

In [33]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_counts[classification_counts < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_sans_EIN_df['CLASSIFICATION'] = application_sans_EIN_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_sans_EIN_df['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64

In [34]:
# Convert categorical data to numeric with `pd.get_dummies`
application_dummies_sans_EIF_df = pd.get_dummies(application_sans_EIN_df).astype(int)
application_dummies_sans_EIF_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,NAME_AIR FORCE ASSOCIATION,NAME_ALABAMA FEDERATION OF WOMENS CLUBS,NAME_ALABAMA TREASURE FOREST ASSOCIATION,NAME_ALPHA PHI SIGMA,NAME_ALPHA PHI SIGMA INC,NAME_ALTRUSA INTERNATIONAL FOUNDATION INC,NAME_AMATEUR ATHLETIC UNION OF THE UNITED STATES,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [35]:
# Split our preprocessed data into our features and target arrays
X = application_dummies_sans_EIF_df.drop(["IS_SUCCESSFUL"], axis='columns').values
y = application_dummies_sans_EIF_df["IS_SUCCESSFUL"].values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [36]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [37]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features_sans_EIN = len(X_train[0])
hidden_nodes_sans_EIN_layer1 =  100
hidden_nodes_sans_EIN_layer2 = 40
hidden_nodes_sans_EIN_layer3 = 10

nn_sans_EIN_model = tf.keras.models.Sequential()

# First hidden layer
nn_sans_EIN_model.add(
    tf.keras.layers.Dense(units=hidden_nodes_sans_EIN_layer1, input_dim=number_input_features_sans_EIN, activation="relu")
)

# Second hidden layer
nn_sans_EIN_model.add(tf.keras.layers.Dense(units=hidden_nodes_sans_EIN_layer2, activation="tanh"))

# Third hidden layer
nn_sans_EIN_model.add(tf.keras.layers.Dense(units=hidden_nodes_sans_EIN_layer3, activation="sigmoid"))

# Output layer
nn_sans_EIN_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_sans_EIN_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               16000     
                                                                 
 dense_7 (Dense)             (None, 40)                4040      
                                                                 
 dense_8 (Dense)             (None, 10)                410       
                                                                 
 dense_9 (Dense)             (None, 1)                 11        
                                                                 
Total params: 20461 (79.93 KB)
Trainable params: 20461 (79.93 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [38]:
# Compile the model
nn_sans_EIN_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [39]:
# Train the model
fit_model = nn_sans_EIN_model.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 457us/step - loss: 0.4969 - accuracy: 0.7610
Epoch 2/100
804/804 [==============================] - 0s 454us/step - loss: 0.4667 - accuracy: 0.7740
Epoch 3/100
804/804 [==============================] - 0s 453us/step - loss: 0.4619 - accuracy: 0.7778
Epoch 4/100
804/804 [==============================] - 0s 453us/step - loss: 0.4581 - accuracy: 0.7784
Epoch 5/100
804/804 [==============================] - 0s 453us/step - loss: 0.4560 - accuracy: 0.7788
Epoch 6/100
804/804 [==============================] - 0s 454us/step - loss: 0.4549 - accuracy: 0.7774
Epoch 7/100
804/804 [==============================] - 0s 453us/step - loss: 0.4530 - accuracy: 0.7794
Epoch 8/100
804/804 [==============================] - 0s 480us/step - loss: 0.4522 - accuracy: 0.7795
Epoch 9/100
804/804 [==============================] - 0s 454us/step - loss: 0.4515 - accuracy: 0.7810
Epoch 10/100
804/804 [==============================] - 0s 452us/step - l

## Looks promising

In [40]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_sans_EIN_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.4668 - accuracy: 0.7725 - 118ms/epoch - 440us/step
Loss: 0.4668086767196655, Accuracy: 0.7724781632423401


In [41]:
# Export our model to HDF5 file
nn_sans_EIN_model.save("Resources/AlphabetSoupCharity_Optimization_3rd_try.h5")

/Users/rolandabellano/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


***
# Conclusion

Overall, by optimizing the model we are able to increase the accuracy to 77.25%. 

This means we are able to correctly classify each of the points in the test data 77.25% of the time. In other words an applicant has a close to 80% chance of being successful if they have the following:

- The NAME of the applicant appears more than <b>20 times (they have applied more than 20 times)</b>
- The type of APPLICATION is one of the following: <b>T3, T4, T5, T6, and surprisingly, T19!</b>
- The application has the following values for CLASSIFICATION: C1000, C1200, C2000,C2100, C3000, and C7000.

-----
#### Alternative Model
An alternative model to recommend is the Random Forest model as it is suited for classification problems. As can be seen below, using the Random Forest model we can achieve SLIGHTLY above 75% accuracy. 

In [42]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [43]:
# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=150, random_state=75) 

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)
print(f" Random forest model accuracy: {accuracy_score(y_test,y_pred):.3f}")

 Random forest model accuracy: 0.759
